In [29]:
import time
import hmac
import requests
import arrow
import pandas as pd
import asyncio
from datetime import datetime as dt
from pprint import pprint
from tqdm import tqdm

In [31]:
print(dt.fromtimestamp(1608249600))
print(dt.fromtimestamp(time.time()-(86400)))

2020-12-18 00:00:00
2021-12-21 12:45:25.424341


In [131]:
def DForginaser(dfname,newname):
    newname = dfname.drop('startTime',axis=1)
    newname['Date'] = [arrow.Arrow.fromtimestamp(date).format('YYYY-MM-DD HH:mm') for date in newname['time']]
    newname.drop('time',axis=1,inplace=True)
    colname = {
                'Date':'Date',
                'open':'Open',
                'high':'High',
                'low':'Low',
                'close':'Close',
                'volume':'Volume'
                }
    newname.rename(columns=colname, inplace=True)
    col_order = ['Date','Open','High','Low','Close','Volume']
    newname = newname[col_order]
    newname.sort_values(by='Date',inplace=True,ascending=True)
    return newname

In [132]:
# window length in seconds. options: 15, 60, 300, 900, 3600, 14400, 86400, or any multiple of 86400 up to 30*86400
def GetHistoFTX(tickers,startdate,enddate,timeframe):
    #date format as arg must be YYYY-MM-DD
    resolution = timeframe 
    for ticker in tickers:
        end_ = arrow.get(enddate).timestamp
        datelimit = arrow.get(startdate).timestamp - resolution #creates a loop limit
        nbline = 1000
        start_ = end_ - (resolution*nbline)
        ticker_df = []
        while start_ > datelimit:
            #print(dt.fromtimestamp(end_),dt.fromtimestamp(start_))
            url = f'https://ftx.com/api/markets/{ticker}/candles'.format()
            params_histo = dict(resolution = resolution,
                    limit =1500,
                    start_time=int(start_),
                    end_time=int(end_))
            r = requests.get(url=url,params=params_histo)
            data = r.json()
            time.sleep(0.1)

            if end_ == data['result'][0]['time']/1000:
                break
            elif data['success'] is 'False':        
                break
            else:
                end_ = round(data['result'][0]['time']/1000,0)

            for i in data['result']:
                ticker_df.append(i)
            
            start_ = end_ - (resolution*nbline)

        ticker_df = pd.DataFrame(ticker_df)
        #clean = pd.concat([ticker_df,pd.DataFrame(clean)])
        filename = [ticker.replace('/','-') if '/' in ticker else ticker][0]
        ticker_df = DForginaser(ticker_df,filename)
        path=r'C:\Users\Anthony\Documents\FTX'
        ticker_df.to_csv(f'{path}\{filename}.csv'.format(),index=False)

In [133]:
tickers = ['ETH/USD','BTC/USD','ETH-PERP','FTM-PERP']
GetHistoFTX(tickers,'2020-12-20','2021-12-20',3600)

In [134]:
df = pd.read_csv('FTM-PERP.csv')

In [135]:
df.shape

(7204, 6)

In [136]:
df.tail()

,Date,Open,High,Low,Close,Volume
7199,2021-12-19 20:00,1.42430,1.43660,1.40930,1.43420,9.580447e+05
7200,2021-12-19 21:00,1.43420,1.43715,1.42280,1.42800,1.002454e+06
7201,2021-12-19 22:00,1.42800,1.45595,1.42315,1.45435,1.496372e+06
7202,2021-12-19 23:00,1.45440,1.45985,1.42285,1.42865,1.902585e+06
7203,2021-12-20 00:00,1.42865,1.43035,1.39270,1.43010,3.639849e+06


In [137]:
df.head()

,Date,Open,High,Low,Close,Volume
0,2021-02-23 04:00,0.428775,0.43910,0.40130,0.42945,385599.69375
1,2021-02-23 05:00,0.429450,0.42945,0.37630,0.41025,548475.39205
2,2021-02-23 06:00,0.410250,0.43485,0.39815,0.40100,189848.88265
3,2021-02-23 07:00,0.401000,0.42295,0.39500,0.41070,309412.26895
4,2021-02-23 08:00,0.410700,0.41950,0.33275,0.33410,440349.72150


In [ ]:
x = ['Close','Volume','avg']
t1 = 24
t2 = 72
for i in test[x]:

    c1 = 'rtn_' + i
    c2 = 'abs_rtn_' + i
    test[c1] = np.log(test[i]/test[i].shift(1))
    test[c2] = abs(np.log(test[i]/test[i].shift(1)))
    c3 = i + '_std' + str(t1)
    c4 = i + '_std' + str(t2) 
    test[c3] = test[i].rolling(window=t1).std()
    test[c4] = test[i].rolling(window=t2).std()
    
    c5 = c2 + '_' + str(t1) + 'MA'
    c6 = c2 + '_' + str(t2) + 'MA'
    test[c5] = test[c2].rolling(window=t1).mean()
    test[c6] = test[c2].rolling(window=t2).mean()
    c7 = i + '_' + str(t1) + 'EMA'
    c8 = i + '_' + str(t2) + 'EMA'
    test[c7] = test[i].ewm(span=t1,adjust=True).mean()
    test[c8] = test[i].ewm(span=t2,adjust=True).mean()
    c9 = i + '_' + str(t1) + 'MA'
    c10 = i + '_' + str(t2) + 'MA'
    test[c9] = test[i].rolling(window=t1).mean()
    test[c10] = test[i].rolling(window=t2).mean()
    
   
    c11 = 'Diff_' + c8 + c7
    test[c11] = test[c8] - test[c7]
        #Bollinger on Close price
    if i == 'Close' or 'avg':
        
        bol1 = ['BolUp'+c9,'BolDn'+c9]
        bol2 = ['BolUp'+c10,'BolDn'+c10]

        test[bol1[0]] = test[i].rolling(window=t1).mean() + (test[c2]*2.5) 
        test[bol1[1]] = test[i].rolling(window=t1).mean() - (test[c2]*2.5)
        test[bol2[0]] = test[i].rolling(window=t2).mean() + (test[c3]*2.5)
        test[bol2[1]] = test[i].rolling(window=t2).mean() - (test[c3]*2.5) 

In [25]:
test = 'ETH-USD'

In [26]:
path=r'C:\Users\Anthony\Documents\FTX'

In [28]:
f'{path}\{test}.csv'.format()

'C:\\Users\\Anthony\\Documents\\FTX\\ETH-USD.csv'